# SQL скрипты для представлений

## new view

### oz_view_ddl

In [ ]:
"""
CREATE OR REPLACE VIEW dds_ir_db.oz_view as
with
oz_view_without_prices_from_orders as (
with 
oz_ord as( 
select 
item_code::varchar,
date::date,
orders::bigint,
account::varchar
from dds_ir_db.dds_oz_orders
where item_code is not null and date is not null and orders > 0
order by date desc, account, item_code::varchar
),
oz_pr as (
select 
item_code::varchar,
date::date,
prices::bigint,
prices_before_discount::bigint,
account::varchar
from dds_ir_db.dds_oz_prices
where item_code is not null and date is not null and prices > 0
order by date desc, account, item_code::varchar
),
oz_st as (
select 
item_code::varchar,
date::date,
fbo_stocks::bigint,
fbs_stocks::bigint,
account::varchar
from dds_ir_db.dds_oz_stocks
where item_code is not null and date is not null and fbo_stocks > 0 or fbs_stocks > 0
order by date desc, account, item_code::varchar
),
oz_buy as (
select
article::varchar as item_code,
date::date,
sum(buyouts::bigint) as buyouts,
'ir' as account
from ir_db.buyouts
where mplace = 'OZON'
group by article::varchar, date::date
having sum(buyouts::bigint) > 0 and date::date <= now()::date
order by date desc, account, article::varchar
)
select 
coalesce(oz_ord.item_code, oz_pr.item_code, oz_st.item_code, oz_buy.item_code) as item_code,
coalesce(oz_ord.date, oz_pr.date, oz_st.date, oz_buy.date) as date,
coalesce(orders, 0) as orders,
prices,
prices_before_discount,
coalesce(fbo_stocks, 0) as fbo_stocks,
coalesce(fbs_stocks, 0) as fbs_stocks,
coalesce(buyouts, 0) as buyouts,
coalesce(orders, 0) - coalesce(buyouts, 0) as orders_without_buyouts,
coalesce(oz_ord.account, oz_pr.account, oz_st.account, oz_buy.account) as account,
'OZ' as marketplace
from 
oz_ord 
full join oz_pr
on oz_ord.item_code = oz_pr.item_code and oz_ord.date = oz_pr.date and oz_ord.account = oz_pr.account
full join oz_st
on oz_pr.item_code = oz_st.item_code and oz_pr.date = oz_st.date and oz_pr.account = oz_st.account
full join oz_buy
on oz_st.item_code = oz_buy.item_code and oz_st.date = oz_buy.date and oz_st.account = oz_buy.account
order by coalesce(oz_ord.date, oz_pr.date, oz_st.date, oz_buy.date)::date desc, 
coalesce(oz_ord.account, oz_pr.account, oz_st.account, oz_buy.account), 
coalesce(oz_ord.item_code, oz_pr.item_code, oz_st.item_code, oz_buy.item_code)::varchar
),
oz_prices_from_full_oz_orders as (
with
oz_prices_from_full_oz_orders_sub
as (
with
oz_pr_from_ozon_orders_copy as (
select 
products_offer_id::varchar as item_code, 
created_at::date as date, 
round(avg(products_price::bigint))::bigint as prices_from_orders,
'ir' as account
from ir_db.ozon_orders_copy
where products_offer_id::varchar is not null 
and created_at::date is not null
and products_price::bigint is not null
and products_price::bigint > 0
group by account, created_at::date, products_offer_id::varchar
having products_offer_id::varchar != ''
order by account, created_at::date desc, products_offer_id::varchar
),
oz_pr_from_oz_fbs_orders as (
select 
products_offer_id::varchar as item_code, 
in_process_at::date as date, 
round(avg(products_price::bigint))::bigint as prices_from_orders,
'ir' as account
from ir_db.oz_fbs_orders
where products_offer_id::varchar is not null 
and in_process_at::date is not null
and products_price::bigint is not null
and products_price::bigint > 0
group by account, in_process_at::date, products_offer_id::varchar
having products_offer_id::varchar != ''
order by account, in_process_at::date desc, products_offer_id::varchar
),
oz_pr_from_oz_kz_fbs_orders as (
select
products_offer_id::varchar as item_code, 
in_process_at::date as date, 
round(avg(to_number(products_price, '99G999D9S')))::bigint as prices_from_orders,
'kz' as account
from ir_db.oz_kz_fbs_orders
where products_offer_id::varchar is not null 
and in_process_at::date is not null
and to_number(products_price, '99G999D9S') is not null
and to_number(products_price, '99G999D9S') > 0
group by account, in_process_at::date, products_offer_id::varchar
having products_offer_id::varchar != ''
order by account, in_process_at::date desc, products_offer_id::varchar
),
oz_pr_from_oz_ssy_fbo_orders as (
select 
offer_id::varchar as item_code, 
created_at::date as date, 
round(avg(price::bigint))::bigint as prices_from_orders,
'ssy' as account
from ir_db.oz_ssy_fbo_orders
where offer_id::varchar is not null 
and created_at::date is not null
and price::bigint is not null
and price::bigint > 0
group by account, created_at::date, offer_id::varchar
having offer_id::varchar != ''
order by account, created_at::date desc, offer_id::varchar
)
select *
from oz_pr_from_ozon_orders_copy
union all 
select *
from oz_pr_from_oz_fbs_orders
union all 
select *
from oz_pr_from_oz_kz_fbs_orders
union all 
select *
from oz_pr_from_oz_ssy_fbo_orders
)
select 
item_code::varchar,
date::date,
round(avg(prices_from_orders))::bigint as prices_from_orders,
account::varchar
from oz_prices_from_full_oz_orders_sub
group by account::varchar, date::date, item_code::varchar
order by account::varchar, date::date desc, item_code::varchar
)
select 
oz_view_without_prices_from_orders.item_code,
oz_view_without_prices_from_orders.date,
oz_view_without_prices_from_orders.orders,
oz_view_without_prices_from_orders.prices,
oz_view_without_prices_from_orders.prices_before_discount,
oz_prices_from_full_oz_orders.prices_from_orders,
oz_view_without_prices_from_orders.fbo_stocks,
oz_view_without_prices_from_orders.fbs_stocks,
oz_view_without_prices_from_orders.buyouts,
oz_view_without_prices_from_orders.orders_without_buyouts,
oz_view_without_prices_from_orders.account,
oz_view_without_prices_from_orders.marketplace
from oz_view_without_prices_from_orders
left join oz_prices_from_full_oz_orders
on oz_view_without_prices_from_orders.account = oz_prices_from_full_oz_orders.account
and oz_view_without_prices_from_orders.date = oz_prices_from_full_oz_orders.date
and oz_view_without_prices_from_orders.item_code = oz_prices_from_full_oz_orders.item_code;
"""

### ya_view_ddl

In [ ]:
"""
CREATE OR REPLACE VIEW dds_ir_db.ya_view as
with
ya_view_without_prices_from_orders as (
with 
ya_ord as( 
select 
item_code::varchar,
date::date,
orders::bigint,
account::varchar
from dds_ir_db.dds_ya_orders
where item_code is not null and date is not null and orders > 0
order by date desc, account, item_code::varchar
),
ya_pr as (
select 
item_code::varchar,
date::date,
prices::bigint,
prices_before_discount::bigint,
account::varchar
from dds_ir_db.dds_ya_prices
where item_code is not null and date is not null and prices > 0
order by date desc, account, item_code::varchar
),
ya_st as (
select 
item_code::varchar,
date::date,
fbo_stocks::bigint,
fbs_stocks::bigint,
account::varchar
from dds_ir_db.dds_ya_stocks
where item_code is not null and date is not null and fbo_stocks > 0 or fbs_stocks > 0
order by date desc, account, item_code::varchar
)
select 
coalesce(ya_ord.item_code, ya_pr.item_code, ya_st.item_code) as item_code,
coalesce(ya_ord.date, ya_pr.date, ya_st.date) as date,
coalesce(orders, 0) as orders,
prices,
prices_before_discount,
coalesce(fbo_stocks, 0) as fbo_stocks,
coalesce(fbs_stocks, 0) as fbs_stocks,
0 as buyouts,
coalesce(orders, 0) as orders_without_buyouts,
coalesce(ya_ord.account, ya_pr.account, ya_st.account) as account,
'YA' as marketplace
from 
ya_ord 
full join ya_pr
on ya_ord.item_code = ya_pr.item_code and ya_ord.date = ya_pr.date and ya_ord.account = ya_pr.account
full join ya_st
on ya_pr.item_code = ya_st.item_code and ya_pr.date = ya_st.date and ya_pr.account = ya_st.account
order by coalesce(ya_ord.date, ya_pr.date, ya_st.date)::date desc, 
coalesce(ya_ord.account, ya_pr.account, ya_st.account), 
coalesce(ya_ord.item_code, ya_pr.item_code, ya_st.item_code)::varchar
),
ya_prices_from_full_ya_orders as (
with
ya_pr_from_yandex_orders2_copy as (
select 
items_offerid::varchar as item_code, 
creationdate::date as date, 
round(avg(items_price::bigint))::bigint as prices_from_orders,
'ir' as account
from ir_db.yandex_orders2_copy
where items_offerid::varchar is not null 
and creationdate::date is not null 
and items_price::bigint is not null 
and items_price::bigint > 0
group by creationdate::date, items_offerid::varchar
order by creationdate::date desc, items_offerid::varchar
)
select * 
from ya_pr_from_yandex_orders2_copy
)
select 
ya_view_without_prices_from_orders.item_code,
ya_view_without_prices_from_orders.date,
ya_view_without_prices_from_orders.orders,
ya_view_without_prices_from_orders.prices,
ya_view_without_prices_from_orders.prices_before_discount,
ya_prices_from_full_ya_orders.prices_from_orders,
ya_view_without_prices_from_orders.fbo_stocks,
ya_view_without_prices_from_orders.fbs_stocks,
ya_view_without_prices_from_orders.buyouts,
ya_view_without_prices_from_orders.orders_without_buyouts,
ya_view_without_prices_from_orders.account,
ya_view_without_prices_from_orders.marketplace
from ya_view_without_prices_from_orders
left join ya_prices_from_full_ya_orders
on ya_view_without_prices_from_orders.account = ya_prices_from_full_ya_orders.account
and ya_view_without_prices_from_orders.date = ya_prices_from_full_ya_orders.date
and ya_view_without_prices_from_orders.item_code = ya_prices_from_full_ya_orders.item_code;
"""

### wb_with_bar_view_ddl

In [11]:
"""
CREATE OR REPLACE VIEW dds_ir_db.wb_with_bar_view as
with 
wb_ord as( 
select 
barcode::bigint,
date::date,
orders::bigint,
account::varchar
from dds_ir_db.dds_wb_orders
where barcode::bigint is not null and date is not null and orders > 0 and barcode::bigint != 0
order by date desc, account, barcode::bigint
),
wb_pr as (
select 
barcode::bigint,
date::date,
prices::bigint,
prices_before_discount::bigint,
account::varchar
from dds_ir_db.dds_wb_prices
where barcode is not null and date is not null and prices > 0 and barcode::bigint != 0
order by date desc, account,barcode::bigint
),
wb_st as (
select 
barcode::bigint,
date::date,
fbo_stocks::bigint,
fbs_stocks::bigint,
account::varchar
from dds_ir_db.dds_wb_stocks
where barcode is not null and date is not null and barcode::bigint != 0
order by date desc, account, barcode::bigint
)
select 
coalesce(wb_ord.barcode, wb_pr.barcode, wb_st.barcode)::bigint as barcode,
coalesce(wb_ord.date, wb_pr.date, wb_st.date) as date,
coalesce(orders, 0) as orders,
prices,
prices_before_discount,
coalesce(fbo_stocks, 0) as fbo_stocks,
coalesce(fbs_stocks, 0) as fbs_stocks,
coalesce(wb_ord.account, wb_pr.account, wb_st.account) as account
from 
wb_ord 
full join wb_pr
on wb_ord.barcode = wb_pr.barcode and wb_ord.date = wb_pr.date and wb_ord.account = wb_pr.account
full join wb_st
on wb_pr.barcode = wb_st.barcode and wb_pr.date = wb_st.date and wb_pr.account = wb_st.account
order by coalesce(wb_ord.date, wb_pr.date, wb_st.date)::date desc, 
coalesce(wb_ord.account, wb_pr.account, wb_st.account), 
coalesce(wb_ord.barcode, wb_pr.barcode, wb_st.barcode)::bigint;
"""

### wb_view_ddl

In [13]:
"""
CREATE OR REPLACE VIEW dds_ir_db.wb_view as
with
wb_view_without_prices_from_orders as (
with
wb_with_art as (
select 
article_name::varchar as item_code,
date::date,
sum(orders)::bigint as orders,
avg(prices)::bigint as prices,
avg(prices_before_discount)::bigint as prices_before_discount,
sum(fbo_stocks)::bigint as fbo_stocks,
sum(fbs_stocks)::bigint as fbs_stocks,
account
from dds_ir_db.wb_with_bar_view as wwbv
left join ir_db.article_barcodes as ab
on wwbv.barcode::bigint = ab.barcode::bigint
where article_name is not null
group by article_name::varchar, date::date, account::varchar
),
wb_buy as (
select
article::varchar as item_code,
date::date,
sum(buyouts::bigint) as buyouts,
'ir' as account
from ir_db.buyouts
where mplace = 'WB'
group by article::varchar, date::date
having sum(buyouts::bigint) > 0 and date::date <= now()::date
order by date desc, account, article::varchar
)
select 
coalesce(wb_with_art.item_code, wb_buy.item_code)::varchar as item_code,
coalesce(wb_with_art.date, wb_buy.date)::date as date,
coalesce(orders, 0) as orders,
prices,
prices_before_discount,
coalesce(fbo_stocks, 0) as fbo_stocks,
coalesce(fbs_stocks, 0) as fbs_stocks,
coalesce(buyouts, 0) as buyouts,
coalesce(orders, 0) - coalesce(buyouts, 0) as orders_without_buyouts,
coalesce(wb_with_art.account, wb_buy.account) as account,
'WB' as marketplace
from wb_with_art 
full join wb_buy
on wb_with_art.item_code = wb_buy.item_code
order by coalesce(wb_with_art.date, wb_buy.date)::date desc, 
coalesce(wb_with_art.account, wb_buy.account), 
coalesce(wb_with_art.item_code, wb_buy.item_code)::varchar
),
wb_prices_from_full_wb_orders as (
with
wb_pr_from_wb_orders2_copy as (
select 
replace(supplierarticle::varchar, '/', '') as item_code,
date::date as date,
round(avg((totalprice / 100) * (100 - discountpercent)))::bigint as prices_from_orders,
'ir' as account
from ir_db.wb_orders2_copy
where replace(supplierarticle::varchar, '/', '') is not null 
and totalprice is not null 
and totalprice::bigint > 0
and coalesce(discountpercent, 0)::bigint <= 100
and date is not null 
group by account, date::date, replace(supplierarticle::varchar, '/', '')
having round(avg((totalprice / 100) * (100 - discountpercent)))::bigint > 0
order by account, date::date desc, replace(supplierarticle::varchar, '/', '')
),
wb_pr_from_wb_kz_orders as (
select 
rtrim(replace(supplierarticle::varchar, '/', ''), 'Q') as item_code,
date::date as date,
round(avg((totalprice / 100) * (100 - discountpercent)))::bigint as prices_from_orders,
'kz' as account
from ir_db.wb_kz_orders
where rtrim(replace(supplierarticle::varchar, '/', ''), 'Q') is not null 
and totalprice is not null 
and totalprice::bigint > 0
and coalesce(discountpercent, 0)::bigint <= 100
and date is not null 
group by account, date::date, rtrim(replace(supplierarticle::varchar, '/', ''), 'Q')
having round(avg((totalprice / 100) * (100 - discountpercent)))::bigint > 0
order by account, date::date desc, rtrim(replace(supplierarticle::varchar, '/', ''), 'Q')
)
select * 
from wb_pr_from_wb_orders2_copy
union all
select *
from wb_pr_from_wb_kz_orders
)
select 
wb_view_without_prices_from_orders.item_code,
wb_view_without_prices_from_orders.date,
wb_view_without_prices_from_orders.orders,
wb_view_without_prices_from_orders.prices,
wb_view_without_prices_from_orders.prices_before_discount,
wb_prices_from_full_wb_orders.prices_from_orders,
wb_view_without_prices_from_orders.fbo_stocks,
wb_view_without_prices_from_orders.fbs_stocks,
wb_view_without_prices_from_orders.buyouts,
wb_view_without_prices_from_orders.orders_without_buyouts,
wb_view_without_prices_from_orders.account,
wb_view_without_prices_from_orders.marketplace
from wb_view_without_prices_from_orders
left join wb_prices_from_full_wb_orders
on wb_view_without_prices_from_orders.account = wb_prices_from_full_wb_orders.account
and wb_view_without_prices_from_orders.date = wb_prices_from_full_wb_orders.date
and wb_view_without_prices_from_orders.item_code = wb_prices_from_full_wb_orders.item_code;
"""

### RC_stocks_view_ddl

In [ ]:
"""
CREATE OR REPLACE VIEW dds_ir_db.RC_stocks_view as
with
RC_stocks as (
with 
Irkutsk_stocks as 
(
select 
Артикул::varchar as item_code,
Дата_обновления::date as date,
sum(coalesce(Сейчас_Доступно::bigint, 0))::bigint as Irkutsk_stocks
from ir_db.ir_wh_stocks
where Склад::varchar = 'Склад Иркутск'
and Артикул::varchar is not null
and Дата_обновления::date is not null
and Сейчас_Доступно::bigint is not null
group by Дата_обновления::date, Артикул::varchar
order by Дата_обновления::date desc, Артикул::varchar
),
Khimki_stocks as 
(
select 
Артикул::varchar as item_code,
Дата_обновления::date as date,
sum(coalesce(Сейчас_Доступно::bigint, 0))::bigint as Khimki_stocks
from ir_db.ir_wh_stocks
where Склад::varchar = 'СкладХимки Ячеистый'
and Артикул::varchar is not null
and Дата_обновления::date is not null
and Сейчас_Доступно::bigint is not null
group by Дата_обновления::date, Артикул::varchar
order by Дата_обновления::date desc, Артикул::varchar
)
select 
coalesce(Irkutsk_stocks.item_code, Khimki_stocks.item_code) as item_code,
coalesce(Irkutsk_stocks.date, Khimki_stocks.date) as date,
Irkutsk_stocks,
Khimki_stocks,
'ir' as account
from Irkutsk_stocks
full join Khimki_stocks
on Irkutsk_stocks.item_code = Khimki_stocks.item_code
and Irkutsk_stocks.date = Khimki_stocks.date
)
select * 
from RC_stocks;
"""

### data_mart_ddl

In [23]:
"""
CREATE MATERIALIZED VIEW IF NOT EXISTS dds_ir_db.data_mart as
with 
data_mart_mat_view as (
select *
from dds_ir_db.oz_view
union all
select * 
from dds_ir_db.wb_view
union all
select * 
from dds_ir_db.ya_view
)
select
ROW_NUMBER() over() as id,
data_mart_mat_view.item_code::varchar(10),
data_mart_mat_view.date::date,
coalesce(orders, 0)::bigint as orders,
prices,
prices_before_discount,
prices_from_orders,
coalesce(fbo_stocks, 0)::bigint as fbo_stocks,
coalesce(fbs_stocks, 0)::bigint as fbs_stocks,
coalesce(buyouts, 0)::bigint as buyouts,
coalesce(orders_without_buyouts, 0)::bigint as orders_without_buyouts,
irkutsk_stocks,
khimki_stocks,
data_mart_mat_view.account::varchar(10),
marketplace::varchar(20)
from data_mart_mat_view
left join dds_ir_db.rc_stocks_view
on data_mart_mat_view.item_code = rc_stocks_view.item_code
and data_mart_mat_view.date = rc_stocks_view.date
and data_mart_mat_view.account = rc_stocks_view.account
order by marketplace, account, date desc, item_code;
create unique index on dds_ir_db.data_mart (id);


REFRESH MATERIALIZED view dds_ir_db.data_mart;
"""

### wb_bar_with_null_art_view_ddl

In [ ]:
"""
CREATE OR REPLACE VIEW dds_ir_db.wb_bar_with_null_art_view as
with
distinct_barcode as (
select distinct barcode::bigint
from
(
select distinct barcode::bigint
from dds_ir_db.dds_wb_orders
union all 
select distinct barcode::bigint
from dds_ir_db.dds_wb_prices
union all
select distinct barcode::bigint
from dds_ir_db.dds_wb_stocks
) as d_b
where barcode is not null and barcode::bigint != 0
)
select 
distinct_barcode.barcode::bigint,
article_name::varchar,
count(*) OVER () as null_art
from distinct_barcode
left join ir_db.article_barcodes as a_b
on distinct_barcode.barcode::bigint = a_b.barcode::bigint
where article_name::varchar is null;
"""

## full_wb

## full_oz

## full_ya

## full_sb

## full_le

## marketplaces_full_data